In [ ]:
import os
from pathlib import Path
import tempfile

import numpy as np
import matplotlib.pyplot as plt

try:
    import cupy as cp
    if cp.cuda.is_available():
        print("GPU is available")
        print("GPU:", cp.cuda.runtime.getDeviceProperties(0)["name"])
    else:
        print("CuPy installed but no GPU detected")
except Exception as e:
    print(f"GPU not available: {e}")
    print("Running on CPU only")

from IPython.display import HTML, display
import h5py
import tdgl
from tdgl.geometry import box, circle
from tdgl.visualization.animate import create_animation

plt.rcParams["figure.figsize"] = (5, 4)
tempdir = tempfile.TemporaryDirectory()



In [ ]:
physical_params = {
    "length_units": "um",
    "xi": 0.1,
    "london_lambda": 2,
    "thickness": 0.1,
    "gamma": 1,
    "width": 0.5,
    "height": 0.5,
    "terminal_size": (0.05, 0.2),
    "hole_radius": 0.05,
}

layer = tdgl.Layer(
    coherence_length=physical_params["xi"],
    london_lambda=physical_params["london_lambda"],
    thickness=physical_params["thickness"],
    gamma=physical_params["gamma"],
)

main_film = tdgl.Polygon(
    "film",
    points=box(physical_params["width"], physical_params["height"]),
).resample(401)
hole = tdgl.Polygon(
    "hole",
    points=circle(physical_params["hole_radius"], center=(0, 0)),
)

term_width, term_height = physical_params["terminal_size"]
source1 = (
    tdgl.Polygon("source1", points=box(term_width, term_height))
    .translate(-physical_params["width"] / 2, 0)
    .buffer(0)
)
drain1 = (
    tdgl.Polygon("drain1", points=box(term_width, term_height))
    .translate(physical_params["width"] / 2, 0)
    .buffer(0)
)

probe_points = [
    ((-physical_params["width"] * 0.25, 0.0), (physical_params["width"] * 0.25, 0.0))
]

device = tdgl.Device(
    "2currents",
    layer=layer,
    film=main_film,
    holes=[hole],
    terminals=[source1, drain1],
    length_units=physical_params["length_units"],
    probe_points=probe_points,
)
fig, ax = device.draw()

In [ ]:
device.make_mesh(max_edge_length=physical_params["xi"] / 4, smooth=100)
display(device.mesh_stats())
fig, ax = device.plot(mesh=True, legend=False)

## Current Sweep
Reuse the previously computed sweep cache (`two_terminal_1gap_sweep`).

In [ ]:
currents = np.linspace(0.0, 12.0, 600)
voltages = []
times = []
solutions = []

sweep_dir = Path("../datasets/two_terminal_1gap_sweep")
sweep_dir.mkdir(exist_ok=True)

options = tdgl.SolverOptions(
    solve_time=300,
    output_file=os.path.join(tempdir.name, "weak-link-1gap-current.h5"),
    field_units="mT",
    current_units="uA",
)
applied_vector_potential = tdgl.sources.constant.ConstantField(
    30, field_units=options.field_units, length_units=device.length_units
)

for idx, bias in enumerate(currents):
    save_path = sweep_dir / f"bias_{idx:04d}.npz"
    if save_path.exists():
        cached = np.load(save_path)
        voltages.append(cached["voltage"])
        times.append(cached["time"])
        solutions.append(None)
        continue

    sweep_solution = tdgl.solve(
        device=device,
        options=options,
        applied_vector_potential=applied_vector_potential,
        terminal_currents=dict(source1=bias, drain1=-bias),
    )

    voltage_trace = sweep_solution.dynamics.voltage()
    time_trace = sweep_solution.dynamics.time

    voltages.append(voltage_trace)
    times.append(time_trace)
    solutions.append(sweep_solution)

    np.savez_compressed(
        save_path,
        current=bias,
        time=time_trace,
        voltage=voltage_trace,
    )

## Field Bias Snapshot
Run one extra bias point for current streamlines.

In [ ]:
zero_current_solution = tdgl.solve(
    device=device,
    options=options,
    applied_vector_potential=applied_vector_potential,
    terminal_currents=dict(source1=2, drain1=-2),
)
fig, ax = zero_current_solution.plot_currents(min_stream_amp=0.075, vmin=6, vmax=10, cmap="turbo")

## I-V Extraction
Reuse `series2` directory for stored traces and plot the averaged I-V curve.

In [ ]:
output_dir = Path("../datasets/series2")
output_dir.mkdir(exist_ok=True)

voltage_values = np.zeros(len(voltages))
plt.figure(figsize=(5, 4), dpi=200)
pair = [0, 1]

for idx in range(len(voltages)):
    vtrace = voltages[idx]
    if vtrace is None and solutions[idx] is not None:
        vtrace = solutions[idx].dynamics.voltage(pair[0], pair[1])
    if vtrace is None:
        continue
    max_idx = vtrace.shape[0] - 1
    voltage_values[idx] = np.mean(vtrace[max_idx])
    data_series = np.vstack((times[idx], vtrace))
    np.save(output_dir / f"output_{idx}.npy", data_series)

plt.plot(currents, voltage_values, color="firebrick")
plt.xlabel("Corriente [$\mu A$]")
plt.ylabel("Voltage [$V_0$]")
plt.xlim(0, 15)
plt.ylim(0, 1.5)

## Voltage Time Traces
Plot selected bias points vs time for diagnostics.

In [ ]:
target_currents = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
indices = [np.abs(currents - val).argmin() for val in target_currents]

plt.figure(figsize=(6, 4))
for idx, val in zip(indices, target_currents):
    vtrace = voltages[idx]
    ttrace = times[idx]
    if vtrace is None or ttrace is None:
        continue
    plt.plot(ttrace, vtrace, label=f"I = {currents[idx]:.2f} \μA")
plt.xlabel("Tiempo")
plt.ylabel("Voltaje [$V_0$]")
plt.title("Evolución temporal del voltaje")
plt.legend(bbox_to_anchor=(1.05, 1), loc="upper left", borderaxespad=0.0)
plt.tight_layout()